In [15]:
import numpy as np
import pandas as pd
import os
import time
import schedule
import joblib
import datetime
import multiprocessing as mp
from get_strategy_meta import get_all_strategy_meta
from query_api import get_path_dict, gen_connection, load_item_by_key, query_mysql
from get_signal import write_signal, get_signal
from mongo_functions import get_portfolio_db

Using TensorFlow backend.


In [16]:
db = get_portfolio_db()

In [17]:
coll = db["SignalOne"]

In [18]:
result = coll.find()

In [19]:
df = pd.DataFrame(result)
df.columns

Index(['_id', 'code', 'key', 'last_date', 'query_date', 'status',
       'status_code', 'write_time', 'y'],
      dtype='object')

In [23]:
df.rename( columns={"code": "a"})

,_id,a,key,last_date,query_date,status,status_code,write_time,y
0,5d19d058a2e0d0ba7c29b099,Z_C007,s_AUDUSD,2019-06-11 04:25:00,2019-07-01 05:20:05,Outdated Signal.,-2,2019-07-01 05:20:24,0.657334
1,5d19d06cd7ff7b912f29b099,Z_C007,s_NZDUSD,2019-06-11 04:25:00,2019-07-01 05:20:05,Outdated Signal.,-2,2019-07-01 05:20:44,0.476752
2,5d19d07f9ec876499629b099,Z_C007,s_XAUUSD,2019-06-11 04:25:00,2019-07-01 05:20:05,Outdated Signal.,-2,2019-07-01 05:21:03,0.423915
3,5d19d093630bb63d2e29b099,Z001,b_XAGUSD,2019-06-11 04:25:00,2019-07-01 05:20:05,Outdated Signal.,-2,2019-07-01 05:21:23,0.297593
4,5d19d0a79a8e7e719229b099,Z001,s_XAGUSD,2019-06-11 04:25:00,2019-07-01 05:20:05,Outdated Signal.,-2,2019-07-01 05:21:43,0.121550
5,5d19d0bb9c7b3e8a9a29b099,Z007,b_EURJPY,2019-06-11 04:25:00,2019-07-01 05:20:05,Outdated Signal.,-2,2019-07-01 05:22:03,0.245684
6,5d19d0ce5b6019c88229b099,Z007,b_NZDUSD,2019-06-11 04:25:00,2019-07-01 05:20:05,Outdated Signal.,-2,2019-07-01 05:22:22,0.166772
7,5d19d0e2bf4798922929b099,Z007,b_XAGUSD,2019-06-11 04:25:00,2019-07-01 05:20:05,Outdated Signal.,-2,2019-07-01 05:22:42,0.390682
8,5d19d0f5929f4a31b029b099,Z007,s_EURJPY,2019-06-11 04:25:00,2019-07-01 05:20:05,Outdated Signal.,-2,2019-07-01 05:23:01,0.035779
9,5d19d109177aed4cf029b099,Z007,s_NZDUSD,2019-06-11 04:25:00,2019-07-01 05:20:05,Outdated Signal.,-2,2019-07-01 05:23:21,0.057034


In [ ]:
df.loc[pd.to_datetime(df["query_date"]) > datetime.datetime(2019, 7, 1, 5, 30)]

In [58]:
connection = gen_connection()
dt =datetime.datetime(2019,7,1)
table = "db_60_min_4"
df = query_mysql(connection, dt, table, window = 1000)

In [59]:
min_col = df.columns
len(min_col)

58

In [60]:
joblib.dump(min_col, filename = "MIN_60_COLUMNS.pkl")

['MIN_60_COLUMNS.pkl']

In [61]:
prices_query = db["prices"].find()
pdf = pd.DataFrame(prices_query)

In [62]:
pdf.head()


,_id,name,price,time
0,5d19d2ac0a2b8939fc01cc48,EURUSD,1.13658,2019-06-25 21:10:00
1,5d19d2ac0a2b8939fc01cc49,USDJPY,107.15700,2019-06-25 21:10:00
2,5d19d2ac0a2b8939fc01cc4a,GBPUSD,1.26817,2019-06-25 21:10:00
3,5d19d2ac0a2b8939fc01cc4b,AUDUSD,0.69576,2019-06-25 21:10:00
4,5d19d2ac0a2b8939fc01cc4c,USDCHF,0.97476,2019-06-25 21:10:00


In [63]:
%%time
pdf2 = pdf.pivot(index='time', columns='name', values='price')

CPU times: user 14.9 ms, sys: 376 µs, total: 15.3 ms
Wall time: 14.4 ms


In [64]:
pdf2[min_col].head()

name,EURUSD,EEM,EFA,EWZ,FXI,GDX,HYG,IAU,IWM,SPY,...,ITB,EWC,UNG,USDJPY,GBPUSD,AUDUSD,USDCAD,EURJPY,NZDUSD,XAGUSD
time,,,,,,,,,,,,,,,,,,,,,
2019-06-25 21:10:00,1.13658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,107.157,1.26817,0.69576,1.31638,121.796,0.66360,NaN
2019-06-25 21:15:00,1.13669,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,107.162,1.26900,0.69577,1.31641,121.805,0.66346,NaN
2019-06-25 21:20:00,1.13671,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,107.166,1.26817,0.69577,1.31677,121.815,0.66336,NaN
2019-06-25 21:25:00,1.13670,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,107.160,1.26880,0.69571,1.31730,121.803,0.66331,NaN
2019-06-25 21:30:00,1.13658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,107.156,1.26819,0.69572,1.31745,121.789,0.66335,NaN


NameError: name 'pdf2' is not defined

In [9]:
import datetime
datetime.datetime(2019,2,2) + datetime.timedelta(days = 0, seconds = 60*60*24*3 + 20)

datetime.datetime(2019, 2, 5, 0, 0, 20)